# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!

In [56]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0710 18:58:47.760281 4713153984 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0710 18:58:47.789806 4713153984 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [57]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [58]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0710 18:58:50.184201 4713153984 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 18:58:50.185960 4713153984 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0710 18:58:50.187906 4713153984 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0710 18:58:50.190210 4713153984 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0710 18:58:50.191877 4713153984 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 18:58:50.193701 4713153984 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [59]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [60]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [61]:
alices_data = data[2:4].send(alice)
alices_target = target[2:4].send(alice)

In [62]:
# Initialize A toy model
model = nn.Linear(2,1)

In [63]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [64]:
bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [65]:
for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_target - bobs_pred)**2).sum()
    bobs_loss.backward()

    # we update the gradients
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(bobs_loss)

    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_target - alices_pred)**2).sum()
    alices_loss.backward()

    # we update the gradients
    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(alices_loss)

tensor(0.2083)
tensor(0.0552)
tensor(0.0321)
tensor(0.0180)
tensor(0.0256)
tensor(0.0087)
tensor(0.0213)
tensor(0.0059)
tensor(0.0177)
tensor(0.0047)
tensor(0.0148)
tensor(0.0039)
tensor(0.0123)
tensor(0.0034)
tensor(0.0102)
tensor(0.0029)
tensor(0.0085)
tensor(0.0024)
tensor(0.0071)
tensor(0.0021)


## Averaging Bob and Alice together

In [66]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [67]:
secure_worker._objects

{79890455925: Parameter containing:
 tensor([[0.0580, 0.5596]], requires_grad=True),
 51131411134: Parameter containing:
 tensor([-0.0357], requires_grad=True),
 8220018012: Parameter containing:
 tensor([[0.1078, 0.7324]], requires_grad=True),
 41803075685: Parameter containing:
 tensor([0.2070], requires_grad=True)}

In [68]:
# Now we average them together
model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())

RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()

In [69]:
model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()

In [70]:
with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

## Rinse and repeat

In [71]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0045) Alice:tensor(0.0002)
Bob:tensor(0.0021) Alice:tensor(8.7555e-05)
Bob:tensor(0.0010) Alice:tensor(0.0002)
Bob:tensor(0.0005) Alice:tensor(0.0003)
Bob:tensor(0.0002) Alice:tensor(0.0003)
Bob:tensor(0.0001) Alice:tensor(0.0003)
Bob:tensor(6.4945e-05) Alice:tensor(0.0003)
Bob:tensor(3.5710e-05) Alice:tensor(0.0003)
Bob:tensor(2.0459e-05) Alice:tensor(0.0002)
Bob:tensor(1.2215e-05) Alice:tensor(0.0002)


In [72]:
preds = model(data)
loss = ((preds - target)**2).sum()

In [73]:
preds

tensor([[0.9502],
        [0.9560],
        [0.0346],
        [0.0405]], grad_fn=<AddmmBackward>)

In [74]:
target

tensor([[1.],
        [1.],
        [0.],
        [0.]], requires_grad=True)

In [75]:
loss.data

tensor(0.0073)

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alternative. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [76]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [77]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [78]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [79]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [80]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [81]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this information. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [82]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [83]:
# try this project here!

In [84]:
import random

In [85]:
Q = 23740629843760239486723

In [86]:
x = 5

In [87]:
def encrypt(x, n_shares=3):
    shares = list()

    for i in range(n_shares-1):
        shares.append(random.randint(0,Q))
        
    final_share = Q - (sum(shares) % Q) + x
    shares.append(final_share)  
    return tuple(shares)

In [88]:
encrypt(x, n_shares=10)

(6171400941859657220126,
 18777588959093668389341,
 14668472655706025212004,
 10930477521510753351389,
 19358849117883315429148,
 7813855241554367738138,
 14271430865504129070255,
 20489267031411603559575,
 23068728511877401517398,
 6893708216160515432969)

In [89]:
def decrypt(shares):
    return sum(shares) % Q

In [90]:
decrypt(shares)

9140981183753532185309

In [91]:
5 /  Q

2.1060940812883077e-22

In [92]:
# what's left from x/Q (le reste)
sum(shares)

9140981183753532185309

In [93]:
# the quotient is = x/Q = 2.1060940812883077e-22 (le quotient)
Q*2.1060940812883077e-22 # the value of the divider (le diviseur) = x

5.0

In [94]:
x/Q

2.1060940812883077e-22

In [95]:
# adding the corresponding shares together
def add(a,b):
    c = list()
    
    assert(len(a) == len(b))
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
        
    return tuple(c)

In [96]:
decrypt(add(encrypt(5), encrypt(10)))

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [97]:
BASE=10
PRECISION=4

In [98]:
# So that we can handle decimal numbers
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

# So we can handle negative numbers 
def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [99]:
encode(3.5)

35000

In [100]:
decode(35000)

3.5

In [101]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [104]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [105]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [106]:
# Splitting x data into shares and then we will point toward that data
x = x.share(bob, alice, secure_worker)

In [108]:
# Using shared gave us different workers for our 3 different shares 
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:54065548245 -> bob:34332448233]
	-> (Wrapper)>[PointerTensor | me:42970058976 -> alice:94611061266]
	-> (Wrapper)>[PointerTensor | me:26172386147 -> secure_worker:61039275338]
	*crypto provider: me*

In [107]:
bob._objects

{34332448233: tensor([4437624463537167401, 1180599325260548884, 3530204341467641754,
         1607728995405283304, 4226742983240251619])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [112]:
y = x + x

In [113]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:80223600479 -> bob:19296064570]
	-> (Wrapper)>[PointerTensor | me:19993381420 -> alice:84936762457]
	-> (Wrapper)>[PointerTensor | me:3636484492 -> secure_worker:97992422151]
	*crypto provider: me*

In [114]:
bob._objects

{34332448233: tensor([4437624463537167401, 1180599325260548884, 3530204341467641754,
         1607728995405283304, 4226742983240251619]),
 19296064570: tensor([8875248927074334802, 2361198650521097768, 7060408682935283508,
         3215457990810566608, 8453485966480503238])}

In [115]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [121]:
x = th.tensor([0.1,0.2,0.3])

In [122]:
x

tensor([0.1000, 0.2000, 0.3000])

In [123]:
x = x.fix_prec()

In [124]:
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [125]:
x.child.child

tensor([100, 200, 300])

In [128]:
type(x)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [129]:
type(x.child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [130]:
type(x.child.child)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [126]:
y = x + x

In [127]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [131]:
x = th.tensor([0.1, 0.2, 0.3])

In [132]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [133]:
y = x + x

In [134]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation